# APIs and Datasets for Scholarly Publications 

Using [lens.org](https://www.lens.org/) API for scholarly data.

## Prerequisites

1. Clone the [GitHub repository](https://github.com/kaust-library/using_lens_org): https://github.com/kaust-library/using_lens_org
1. Create your virtual environment: `python -m venv venv`.
1. Activate your environment: `. .\venv\Scripts\activate`. (Windows platform) or `. venv/bin/activate` (Linux)
1. Install the required packages: `pip install -r requirements.txt`.

## Loading the Packages and Env File

Load the packages

In [58]:
import dotenv as DE
import os as OS
import requests as RQ
import pprint as PP
import json as JN
import csv as CSV

You may need to create a `.env` file with your _token_ on _root_ directory of your project

```
(venv) PS C:\Users\garcm0b\Work\lens_org> cat .env
MY_TOKEN=(...)
(venv) PS C:\Users\garcm0b\Work\lens_org>
```

Make sure that your `.env` file is in the `.gitignore` file, so we will not upload your credential by accident.

In [59]:
DE.load_dotenv()
api_passwd = OS.environ['MY_TOKEN']

## Examples

### Simple Search

Using the [`requests`](https://docs.python-requests.org/en/latest/index.html) library to test the Lens.org API. We use a singple example from the Swager API test page. Here we see simple query with some fields:

* [Query](https://docs.api.lens.org/request-scholar.html#terms-query): operates in a single term and search for _exact_ term in the field provided.
* [Match](https://docs.api.lens.org/request-scholar.html#match-query): the main use case of the match query is full-text search. It matches each words separately.
* [From/Size](https://docs.api.lens.org/request-scholar.html#offsetsize-based-pagination): use parameter `from` to define the offset and `size` to specify number of records expected.
* [Include/Exclude](https://docs.api.lens.org/request-scholar.html#projection): only request specific fields from the API endpoint.
* [Sort](https://docs.api.lens.org/request-scholar.html#sorting): result can be retrieved in ascending or descending order.
* [Scroll/Scroll_id](https://docs.api.lens.org/request-scholar.html#cursor-based-pagination): You can specify records per page using `size` (default 20 and max 1000) and context alive time `scroll` (default 1 minute). You will receive a `scroll_id` in response, which should be passed via request body to access next set of results.

In [60]:
headers= {"Authorization": api_passwd, "Content-Type": "application/json"}

payload = '''
{
  "query": {
    "match": {
      "title": "Malaria"
    }
  },
  "size": 5,
  "from": 0,
  "include": [
    "title",
    "lens_id",
    "patent_citations_count"
  ],
  "sort": [
    {
      "created": "desc"
    },
    {
      "year_published": "asc"
    }
  ],
  "exclude": null,
  "scroll": null,
  "scroll_id": null
}
'''

rr = RQ.post('https://api.lens.org/scholarly/search', data=payload, headers=headers)

After the query, we check if our request was successful or not by checking the `status_code`. The value `200` means a valid answer from the server, and [any other value](https://docs.api.lens.org/getting-started.html#http-responses) means an error. Next we print the result of the query:

In [61]:
if rr.status_code == 200:
    print(f"Your request was successfull")
    PP.pprint(rr.text)
else:
    print(f"Something went wrong. The return code was '{rr.status_code}'")

Your request was successfull
('{"total":107644,"data":[{"lens_id":"189-780-393-989-766","title":"A world '
 'free of malaria: It is time for Africa to actively champion and take '
 'leadership of elimination and eradication '
 'strategies"},{"lens_id":"085-144-192-014-163","title":"Reflections from the '
 'first South Sudan malaria '
 'conference"},{"lens_id":"091-205-353-493-016","title":"ASYMPTOMATIC MALARIA '
 'INFECTION AND ANAEMIA AMONG SECONDARY SCHOOL CHILDREN IN IPOGUN, ONDO STATE, '
 'NIGERIA"},{"lens_id":"007-476-373-861-646","title":"EFFECT OF MALARIA ON '
 'VISUAL ACUITY (V.A)"},{"lens_id":"027-325-084-244-85X","title":"Peer Review '
 '#2 of Cohesin is involved in transcriptional repression of stage-specific '
 'genes in the human malaria parasite"}],"results":5}')


### Saving to CSV File

The next example we query for the articles with abstract, and we save the output to a CSV file. This can be further expanded by tokenizing the abstract, and using tokens for Machine Learning.

In [62]:
payload = '''{
     "query": {
        "bool": {
            "must": [
                {
                    "query_string": {
                        "query": "catalyzed",
                        "fields": [
                            "title",
                            "abstract",
                            "full_text"
                        ],
                        "default_operator": "or"
                    }
                }
            ],
            "filter": [
                {
                    "term": {
                        "has_abstract": true
                    }
                }
            ]
        }
    },
     "size": 10
}
'''

In [63]:
rr = RQ.post('https://api.lens.org/scholarly/search', data=payload, headers=headers)

The function below is to extract the first and last name of the author(s). The problem is that in the answer, the authors is a [structure with several fields](https://docs.api.lens.org/response-scholar.html#author), like affiliations, ids, initials, etc. Here we just want the name, and in the case of more than one author, we use a different character (`;`) so we don't mix with commas separating the fields.

In [64]:
def get_authors(count: int, aulist: list) -> str:
    """
    Return the author's first and lastname.
    """
    
    if count == 1:
        return aulist[0]['first_name'] + " " + aulist[0]['last_name']
    else:
        names = ""
        for aa in aulist:
            names += aa['first_name'] + " " + aa['last_name'] + "; "
        # hack: remove the last '; '.
        names = names[:-2]
    
        return names

We use the method [`loads`](https://docs.python.org/3/library/json.html#json.loads) to read the output from our request into a JSON object. Next we will save the JSON items as a CSV file. To write the CSV file we'll use the [DictWriter](https://docs.python.org/3.10/library/csv.html#csv.DictWriter) method.

In [65]:
text = JN.loads(rr.text)
data = text['data']

fields = ['lens_id', 'title', 'year_published', 'authors', 'abstract']
row_csv = []

for dd in data:
    row = {}
    for ff in fields:
        row.update({ff: dd[ff]})
    # print(dd['author_count'])
    row['authors'] = get_authors(dd['author_count'], dd['authors'])
    row_csv.append(row)

with open('metadata.csv', "w", newline="", encoding='utf-8') as csvfile:
    writer = CSV.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(row_csv)

Let's give an example of authors as given by the API and after our function:

In [66]:
print("First the output from the API:")
PP.pprint(dd['authors'])
print("\nAfter the function 'get_authors'")
PP.pprint(row['authors'])

First the output from the API:
[{'affiliations': [],
  'first_name': 'Andrei K.',
  'ids': [{'type': 'magid', 'value': '1890584584'}],
  'initials': 'AK',
  'last_name': 'Yudin'},
 {'affiliations': [],
  'first_name': 'John F.',
  'ids': [{'type': 'magid', 'value': '2091457642'}],
  'initials': 'JF',
  'last_name': 'Hartwig'}]

After the function 'get_authors'
'Andrei K. Yudin; John F. Hartwig'


### Building a Query

Next we will build a search. We start querying for KAUST output with the following fields: _title_, _lens\_id_, and _year of publication_.

In [67]:
payload = '''{
    "query": {
        "match_phrase": {
            "author.affiliation.name": "King Abdullah University of Science and Technology"
        }
    },
    "include": [
        "title",
        "lens_id",
        "year_published"
    ],
    "size": 10
}
'''

In [68]:
rr = RQ.post('https://api.lens.org/scholarly/search', data=payload, headers=headers)

In [69]:
if rr.status_code == 200:
    print(f"Your request was successfull")
    PP.pprint(rr.text)
else:
    print(f"Something went wrong. The return code was '{rr.status_code}'")

Your request was successfull
('{"total":26456,"data":[{"lens_id":"015-043-967-234-283","title":"Damp '
 'heat-stable perovskite solar cells with tailored-dimensionality 2D/3D '
 'heterojunctions.","year_published":2022},{"lens_id":"042-815-052-361-010","title":"Nitrogen '
 'Reduction to Ammonia by a Phosphorus-Nitrogen PN\\n            '
 '<sup>3</sup>\\n            P-Mo(V) Nitride Complex: Significant Enhancement '
 'via Ligand '
 'Postmodification","year_published":2023},{"lens_id":"044-368-649-773-289","title":"Canonical '
 'strigolactones are not the major determinant of tillering but important '
 'rhizospheric signals in '
 'rice.","year_published":2022},{"lens_id":"083-564-790-962-620","title":"Metal-Organic '
 'Frameworks in Mixed-Matrix Membranes for High-Speed Visible-Light '
 'Communication.","year_published":2022},{"lens_id":"109-516-570-346-389","title":"Monolithic '
 'Perovskite/Silicon Tandems with >28% Efficiency: Role of Silicon‐Surface '
 'Texture on Perovskite '
 'Pro

Next we include a range for the year of publication.

In [136]:
payload = '''{
    "query": {
        "bool": {
            "must": [
                {
                    "match_phrase":{
                        "author.affiliation.name": "King Abdullah University of Science and Technology"
                    }
                }, {
                    "range": {
                        "year_published": {
                            "gte": "2018",
                            "lte": "2020"
                        }
                    }                
                }
            ],
            "filter": [
                {
                    "term": {
                        "publication_type": "journal article"
                    }
                }
            ]
        }
    },
    "include": [
        "lens_id",
        "title",
        "year_published"
    ],
    "size": 10
}
'''

In [137]:
rr = RQ.post('https://api.lens.org/scholarly/search', data=payload, headers=headers)

In [138]:
if rr.status_code == 200:
    print(f"Your request was successfull")
    PP.pprint(rr.text)
else:
    print(f"Something went wrong. The return code was '{rr.status_code}'")

Your request was successfull
('{"total":5895,"data":[{"lens_id":"008-897-357-875-011","title":"Quantum Dots '
 'Supply Bulk- and Surface-Passivation Agents for Efficient and Stable '
 'Perovskite Solar '
 'Cells","year_published":2019},{"lens_id":"024-196-058-429-896","title":"Halogen '
 'Vacancies Enable Ligand‐Assisted Self‐Assembly of Perovskite Quantum Dots '
 'into '
 'Nanowires","year_published":2019},{"lens_id":"034-562-241-096-057","title":"Managing '
 'grains and interfaces via ligand anchoring enables 22.3%-efficiency inverted '
 'perovskite solar '
 'cells","year_published":2020},{"lens_id":"056-388-544-861-841","title":"Intrinsic '
 'efficiency limits in low-bandgap non-fullerene acceptor organic solar '
 'cells","year_published":2020},{"lens_id":"060-777-685-250-677","title":"Correction: '
 'Topology meets MOF chemistry for pore-aperture fine tuning: ftw-MOF platform '
 'for energy-efficient separations via adsorption kinetics or molecular '
 'sieving.","year_published":20